In [1]:
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool

In [2]:
from src.config.config import OPENAI_API_KEY, TAVILY_API_KEY, EXA_API_KEY, PINECONE_API_KEY, LLM_MODEL
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["EXA_API_KEY"] = EXA_API_KEY
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["LLM_MODEL"] = LLM_MODEL

## Handoff Tools

In [3]:

# --- Handoff Tools ---
transfer_to_course_discovery_agent = create_handoff_tool(
    agent_name="course_discovery_agent",
    description="Transfer user to the course discovery agent."
)
transfer_to_course_suitability_agent = create_handoff_tool(
    agent_name="course_suitability_agent",
    description="Transfer user to the course suitability agent."
)
transfer_to_career_path_agent = create_handoff_tool(
    agent_name="career_path_agent",
    description="Transfer user to the career path agent."
)
transfer_to_student_profile_agent = create_handoff_tool(
    agent_name="student_profile_agent",
    description="Transfer user to the student profile agent."
)


## Tools
### 1. Course Discovery and Recommendation Agent Tools

In [4]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from pydantic import BaseModel, Field
from langchain_community.tools.tavily_search import TavilySearchResults

retriever = PineconeVectorStore.from_existing_index(index_name="course-index", embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY)).as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "pinecone_search",
    "A tool to search the Pinecone vector database for relevant course information.",
)


class TavilySearchInput(BaseModel):
    query: str = Field(..., description="The query to search the web for relevant course information.")

class PineconeSearchInput(BaseModel):
    query: str = Field(..., description="The query to search the Pinecone vector database for relevant course information.")


course_discovery_agent_tools=[
    Tool(
        name="tavily_search",
        func=TavilySearchResults(max_results=5),
        description="A tool to search the web for relevant course information.",
        args_schema=TavilySearchInput
    ),
    retriever_tool
]

### 2. Career Path Agent Tools Implementation

In [5]:
# --- Career Path Agent Tools Implementation

llm = ChatOpenAI(model=LLM_MODEL, temperature=0.4)

class CareerPathAnalysisInput(BaseModel):
    query: str = Field(..., description="The query to analyze the career path.")

def ask_llm(query: str) -> str:
    return llm.invoke(query)

career_path_agent_tools=[
    Tool(
        name="career_path_analysis",
        func=ask_llm,
        description="A tool to give insights into the career path as per selected courses or user's ask in the query",
        args_schema=CareerPathAnalysisInput
    )
]

### 3. Course Suitability Agent Tools Implementation

In [6]:

# --- Define Input Schema for the tool ---
class CourseValidationInput(BaseModel):
    course: dict = Field(..., description="Metadata of the course to validate")
    profile: dict = Field(..., description="Student profile dictionary")


import re
def validate_course_tool(course: dict, profile: dict) -> dict:
    validation = {
        "score": 0,
        "matches": [],
        "mismatches": [],
        "questions": []
    }

    # Education level check
    if profile.get("education_level") and profile["education_level"].lower() in course.get("level", "").lower():
        validation["score"] += 1
        validation["matches"].append(f"Education level ({course.get('level')}) matches your current level.")
    else:
        validation["mismatches"].append(f"Course level ({course.get('level')}) might not match your education level.")
        validation["questions"].append(f"Are you comfortable with a course at {course.get('level')} level?")

    # Mode preference check
    if profile.get("preferred_mode") and course.get("mode") and course["mode"].lower() == profile["preferred_mode"].lower():
        validation["score"] += 1
        validation["matches"].append(f"Delivery mode ({course['mode']}) matches your preference.")
    else:
        validation["mismatches"].append(f"Course mode ({course.get('mode')}) differs from your preferred mode.")
        validation["questions"].append(f"Would you consider a {course.get('mode')} course?")

    # Time commitment check
    if "availability" in profile and profile["availability"]:
        try:
            daily_commitment = course.get("daily_commitment", "")
            hours = int(re.search(r"\d+", daily_commitment).group())
            available_hours = int(profile["availability"].get("hours_per_day", 24))
            if hours <= available_hours:
                validation["score"] += 1
                validation["matches"].append("Time commitment fits your availability.")
            else:
                validation["mismatches"].append(f"Required time ({daily_commitment}) might exceed your availability.")
                validation["questions"].append("Can you accommodate this time commitment?")
        except Exception:
            validation["questions"].append(f"Can you commit {course.get('daily_commitment')}?")

    # Prerequisites check
    if course.get("prerequisites"):
        validation["questions"].append("Do you meet these prerequisites: " + ", ".join(course["prerequisites"]))

    # Career goals alignment check
    if profile.get("career_goals") and course.get("career_outcomes"):
        matched_goals = [goal for goal in profile["career_goals"]
                        if any(goal.lower() in outcome.lower() for outcome in course["career_outcomes"])]
        if matched_goals:
            validation["score"] += 1
            validation["matches"].append("Course aligns with your career goals.")

    # Format a readable message summarizing validation
    message_lines = [f"\nRegarding {course.get('title')} by {course.get('provider')}:"]
    if validation["matches"]:
        message_lines.append("\nPositive matches:")
        message_lines.extend([f"✓ {m}" for m in validation["matches"]])
    if validation["mismatches"]:
        message_lines.append("\nPotential concerns:")
        message_lines.extend([f"! {m}" for m in validation["mismatches"]])
    if validation["questions"]:
        message_lines.append("\nQuestions to consider:")
        message_lines.extend([f"? {q}" for q in validation["questions"]])

    message = "\n".join(message_lines)

    return {
        "validation": validation,
        "message": message
    }


course_suitability_agent_tools=[
    Tool(
        name="validate_course_tool",
        func=validate_course_tool,
        description="Validate course suitability against a student's profile and provide matches, mismatches, and clarifying questions.",
        args_schema=CourseValidationInput
    )
]

### 4. Student Profile Agent Tools Implementation

In [7]:
## StudentProfile schema

from typing import List, Optional, Dict
from pydantic import BaseModel

class StudentProfile(BaseModel):
    name: str = Field(..., description="Full name of the student.")
    educational_level: str = Field(..., description="Current level of education, e.g., High School, Undergraduate, etc.")
    age: int = Field(..., description="Age of the student.")
    course_description: str = Field(..., description="A short description of the course the student is looking for.")
    course_mode: str = Field(..., description="Preferred mode of learning: online, offline, hybrid, or any.")
    daily_hours: int = Field(..., description="Number of hours the student can dedicate to learning each day.")
    preferred_timing: str = Field(..., description="Preferred time of the day to take the course: morning, afternoon, or evening.")
    max_duration_months: int = Field(..., description="Maximum duration (in months) the student is willing to commit to the course.")
    language: List[str] = Field(default_factory=lambda: ["English"], description="Preferred languages for the course content.")
    certification_needed: bool = Field(default=False, description="Whether the student wants a certification after completing the course.")
    location_preference: str = Field(default="any", description="Location preference for the course: online, offline, hybrid, or any.")

In [8]:
# Input schema for extract_student_profile tool
from typing import List, Dict, Optional, Any
import json
from pathlib import Path

from pydantic import BaseModel, Field

profile_json_path = Path("./profile.json")

class ExtractProfileInput(BaseModel):
    conversation: str = Field(
        ..., description="Flattened conversation between student and agent as a single string."
    )


def extract_student_profile(conversation: str) -> dict:
    """Extract structured student profile data from plain conversation text."""
    prompt = (
        "You are an expert profile extractor. Extract the following fields from the conversation below.\n"
        "Return a valid JSON with the fields (use null/empty values where not mentioned):\n"
        "- name\n"
        "- educational_level\n"
        "- age\n"
        "- course_description (as a detailed string)\n"
        "- course_mode\n"
        "- daily_hours\n"
        "- preferred_timing\n"
        "- max_duration_months\n"
        "- language (as list)\n"
        "- certification_needed (true/false)\n"
        "- location_preference\n\n"
        "Conversation:\n"
        f"{conversation}"
    )
    structured_llm = llm.with_structured_output(StudentProfile)
    partial_profile_response = structured_llm.invoke([{"role": "user", "content": prompt}])

    try:
        new_profile = json.loads(partial_profile_response.content)
        new_profile = StudentProfile(**profile).dict()
    except Exception:
        new_profile = None

    
    new_profile = partial_profile_response.dict()

    # Save to file
    with profile_json_path.open("r", encoding="utf-8") as f:
        profile = json.load(f)

    # Newly extract fields(new_profile) add to profile, given that add only null/empty fields
    # Merge: only update missing fields
    for key, value in new_profile.items():
        if (
            key in profile and (
                profile[key] is None or
                profile[key] == "" or
                (isinstance(profile[key], list) and not profile[key])
            )
        ):
            profile[key] = value

    # Save updated profile
    with profile_json_path.open("w", encoding="utf-8") as f:
        json.dump(profile, f, indent=2)

    total_fields = len(profile)
    filled_fields = sum(1 for v in profile.values() if v not in [None, "","null", [], {}])
    profile["completion_percent"] = round((filled_fields / total_fields) * 100)

    return profile


# Tool

class EmptyInput(BaseModel):
    description: str = Field(..., description="Description of tool action")


def check_profile_completeness(description: str) -> Dict[str, Any]:
    """Check what percentage of the profile is completed by reading from file."""
    required_fields = [
        "name", "educational_level", "age", "course_description", "course_mode",
        "daily_hours", "preferred_timing", "max_duration_months", "language",
        "certification_needed", "location_preference"
    ]
    
    # Load the profile from file
    with profile_json_path.open("r", encoding="utf-8") as f:
        profile = json.load(f)
    
    total_fields = len(required_fields)
    filled_fields = sum(
        1 for field in required_fields 
        if profile.get(field) not in [None, "", [], {}]
    )

    percent_complete = round((filled_fields / total_fields) * 100)
    return {
        "percent_complete": percent_complete,
        "is_complete": percent_complete == 100
    }



def determine_next_missing_field(description: str) -> Optional[str]:
    """Determine the next field to ask about based on missing data."""
    ordered_fields = [
        "name", "educational_level", "age", "course_description", "course_mode",
        "daily_hours", "preferred_timing", "max_duration_months", "language",
        "certification_needed", "location_preference"
    ]

     # Load the profile from file
    with profile_json_path.open("r", encoding="utf-8") as f:
        profile = json.load(f)

    for field in ordered_fields:
        value = profile.get(field)
        if value in [None, "", [], {}]:
            return field  # Return the first missing field

    return None  # All fields are filled

student_profile_agent_tools=[
    Tool(
        name="extract_student_profile",
        func=extract_student_profile,
        description="Extract structured student profile data from a conversation chat history.",
        args_schema=ExtractProfileInput
    ),
    Tool(
        name="check_profile_completeness",
        func=check_profile_completeness,
        description="Check how much percentage fields are completed in the student profile.",
        args_schema=EmptyInput
    ),
    Tool(
        name="determine_next_missing_field",
        func=determine_next_missing_field,
        description="Determine the next missing profile field to ask the student/user",
        args_schema=EmptyInput
    )
]


## Agents

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model=LLM_MODEL)

In [10]:
# Adding handoff tools to tools

student_profile_agent_tools.append(transfer_to_course_discovery_agent)
course_discovery_agent_tools.append(transfer_to_course_suitability_agent)
course_suitability_agent_tools.append(transfer_to_career_path_agent)
career_path_agent_tools.append(transfer_to_student_profile_agent)


In [11]:

# --- Agent Declarations (strictly following tutorial style) ---
student_profile_agent = create_react_agent(
    model=model,
    tools=student_profile_agent_tools,
    prompt=(
        "You are a student profiling agent. Your job is to interact with students "
        "to gather their student profile. Ask questions to extract each student profile field\n"
       
        "Use tools like `extract_student_profile` to extract info from chat history, "
        "and `determine_missing_field` to guide what to ask next. Once the profile is complete, "
        "you must indicate that the handoff to the recommendation agent can happen."
    ),
    name="student_profile_agent"
)

course_discovery_agent = create_react_agent(
    model=model,
    tools=course_discovery_agent_tools,
    prompt=(
        "You are a course discovery agent. Recommend relevant courses (school, college, or online) "
        "based on the student's profile. Use tools to switch to other agents if needed."
    ),
    name="course_discovery_agent"
)

course_suitability_agent = create_react_agent(
    model=model,
    tools=course_suitability_agent_tools,
    prompt=(
        "You are a suitability analysis agent. Evaluate how suitable a given course is for the student, "
        "based on their profile, constraints, and preferences. "
        "Ask clarifying questions if needed. "
        "If the user requires career advice, handoff to the career advisor agent."
    ),
    name="course_suitability_agent"
)

career_path_agent = create_react_agent(
    model=model,
    tools=career_path_agent_tools,
    prompt=(
        "You are a career advisor. Provide insights into potential career paths aligned with the student's "
        "profile and selected courses. Offer guidance on next steps, degrees, and skill-building."
    ),   
    name="career_path_agent"
)

## Swarm

In [12]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore

# short-term memory
checkpointer = InMemorySaver()
# long-term memory
store = InMemoryStore()

In [13]:
from langgraph_swarm import create_swarm
from langgraph_swarm import SwarmState

# --- Swarm Creation ---
swarm = create_swarm(
    agents=[
        student_profile_agent,
        course_discovery_agent,
        course_suitability_agent,
        career_path_agent
    ],
    default_active_agent="student_profile_agent",
).compile(
    checkpointer=checkpointer,
    store=store
)

# # --- Example User Message ---
# user_message = {
#     "messages": [
#         {
#             "role": "user",
#             "content": (
#                 "I'm a high school student interested in computer science. "
#                 "I want to find a good online course for 6 months that fits my evening schedule."
#             )
#         }
#     ]
# }

# # --- Run the Swarm and Print the Output ---
# for chunk in swarm.stream(user_message):
#     print(chunk)
#     print("\n")


In [14]:
# Delete
store.delete(("swarm",), "1")
checkpointer.delete_thread("1")

In [15]:

profile_json_path = Path("./profile.json")

def reset_profile_to_empty():
    if not profile_json_path.exists():
        print("profile.json not found.")
        return

    with profile_json_path.open("r", encoding="utf-8") as f:
        profile = json.load(f)

    def empty_value(val):
        if isinstance(val, str):
            return ""
        elif isinstance(val, bool):
            return False
        elif isinstance(val, int) or isinstance(val, float):
            return None
        elif isinstance(val, list):
            return []
        elif val is None:
            return None
        else:
            return None

    for key in profile.keys():
        profile[key] = empty_value(profile[key])

    with profile_json_path.open("w", encoding="utf-8") as f:
        json.dump(profile, f, indent=2)
    print("Profile reset to empty values.")

In [16]:
reset_profile_to_empty()

Profile reset to empty values.


In [17]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, ToolCall

def pretty_print_turn(turn, turn_number=None):
    if turn_number is not None:
        print(f"\n=== Turn {turn_number} ===")
    else:
        print("\n=== Conversation Turn ===")

    for msg in turn.get("messages", []):
        if isinstance(msg, HumanMessage):
            print(f"🧑 Human: {msg.content}")
        elif isinstance(msg, AIMessage):
            print(f"🤖 AI: {msg.content}")
            tool_calls = msg.tool_calls or []
            for call in tool_calls:
                print(f"🔧 Tool Call -> ID: {call['id']} | Name: {call['name']} | Args: {call['args']}")
        elif isinstance(msg, ToolMessage):
            print(f"🛠️ Tool Response -> ID: {msg.tool_call_id} | Result: {msg.content}")
        else:
            print(f"📦 Other: {msg.type} -> {msg.content}")

    print("=" * 40)

In [18]:
config = {"configurable": {"thread_id": "1"}}
turn_1 = swarm.invoke(
    {"messages": [{"role": "user", "content": "Hi"}]},
    config=config,
)
# print(turn_1["messages"][-1].content)

pretty_print_turn(turn_1)


=== Conversation Turn ===
🧑 Human: Hi
🤖 AI: Hello! How can I assist you today? Are you looking for information on courses or programs?


In [19]:
turn_2 = swarm.invoke(
    {"messages": [{"role": "user", "content": "I'm Tejas Gadi, 20 years old, a high school student interested in Maths "}]},
    config,
)
# print(turn_2["messages"][-1].content)

pretty_print_turn(turn_2)

/var/folders/hq/tbxvw3vn6wb7ky65zh2vrfm40000gn/T/ipykernel_30364/1747705358.py:45: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  new_profile = partial_profile_response.dict()



=== Conversation Turn ===
🧑 Human: Hi
🤖 AI: Hello! How can I assist you today? Are you looking for information on courses or programs?
🧑 Human: I'm Tejas Gadi, 20 years old, a high school student interested in Maths 
🤖 AI: 
🔧 Tool Call -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Name: extract_student_profile | Args: {'conversation': "I'm Tejas Gadi, 20 years old, a high school student interested in Maths."}
🔧 Tool Call -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Name: check_profile_completeness | Args: {'description': "Check the completeness of Tejas's student profile."}
🛠️ Tool Response -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Result: {"name": "Tejas Gadi", "educational_level": "High School", "age": 20, "course_description": "", "course_mode": "", "daily_hours": 0, "preferred_timing": "", "max_duration_months": 0, "language": [], "certification_needed": false, "location_preference": "", "completion_percent": 55}
🛠️ Tool Response -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Result: {"percent_complete":

## Now From turn 3 do Student Profile Data Collection

In [20]:
turn_3 = swarm.invoke(
    {"messages": [{"role": "user", "content": "I am looking for Linear Algebra courses"}]},
    config,
)
# print(turn_2["messages"][-1].content)

pretty_print_turn(turn_3)

/var/folders/hq/tbxvw3vn6wb7ky65zh2vrfm40000gn/T/ipykernel_30364/1747705358.py:45: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  new_profile = partial_profile_response.dict()



=== Conversation Turn ===
🧑 Human: Hi
🤖 AI: Hello! How can I assist you today? Are you looking for information on courses or programs?
🧑 Human: I'm Tejas Gadi, 20 years old, a high school student interested in Maths 
🤖 AI: 
🔧 Tool Call -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Name: extract_student_profile | Args: {'conversation': "I'm Tejas Gadi, 20 years old, a high school student interested in Maths."}
🔧 Tool Call -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Name: check_profile_completeness | Args: {'description': "Check the completeness of Tejas's student profile."}
🛠️ Tool Response -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Result: {"name": "Tejas Gadi", "educational_level": "High School", "age": 20, "course_description": "", "course_mode": "", "daily_hours": 0, "preferred_timing": "", "max_duration_months": 0, "language": [], "certification_needed": false, "location_preference": "", "completion_percent": 55}
🛠️ Tool Response -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Result: {"percent_complete":

In [21]:
turn_2 = swarm.invoke(
    {"messages": [{"role": "user", "content": "online is preferable"}]},
    config,
)
# print(turn_2["messages"][-1].content)

pretty_print_turn(turn_2)

/var/folders/hq/tbxvw3vn6wb7ky65zh2vrfm40000gn/T/ipykernel_30364/1747705358.py:45: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  new_profile = partial_profile_response.dict()



=== Conversation Turn ===
🧑 Human: Hi
🤖 AI: Hello! How can I assist you today? Are you looking for information on courses or programs?
🧑 Human: I'm Tejas Gadi, 20 years old, a high school student interested in Maths 
🤖 AI: 
🔧 Tool Call -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Name: extract_student_profile | Args: {'conversation': "I'm Tejas Gadi, 20 years old, a high school student interested in Maths."}
🔧 Tool Call -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Name: check_profile_completeness | Args: {'description': "Check the completeness of Tejas's student profile."}
🛠️ Tool Response -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Result: {"name": "Tejas Gadi", "educational_level": "High School", "age": 20, "course_description": "", "course_mode": "", "daily_hours": 0, "preferred_timing": "", "max_duration_months": 0, "language": [], "certification_needed": false, "location_preference": "", "completion_percent": 55}
🛠️ Tool Response -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Result: {"percent_complete":

In [22]:
turn_2 = swarm.invoke(
    {"messages": [{"role": "user", "content": "evenings"}]},
    config,
)
# print(turn_2["messages"][-1].content)

pretty_print_turn(turn_2)

/var/folders/hq/tbxvw3vn6wb7ky65zh2vrfm40000gn/T/ipykernel_30364/1747705358.py:45: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  new_profile = partial_profile_response.dict()



=== Conversation Turn ===
🧑 Human: Hi
🤖 AI: Hello! How can I assist you today? Are you looking for information on courses or programs?
🧑 Human: I'm Tejas Gadi, 20 years old, a high school student interested in Maths 
🤖 AI: 
🔧 Tool Call -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Name: extract_student_profile | Args: {'conversation': "I'm Tejas Gadi, 20 years old, a high school student interested in Maths."}
🔧 Tool Call -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Name: check_profile_completeness | Args: {'description': "Check the completeness of Tejas's student profile."}
🛠️ Tool Response -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Result: {"name": "Tejas Gadi", "educational_level": "High School", "age": 20, "course_description": "", "course_mode": "", "daily_hours": 0, "preferred_timing": "", "max_duration_months": 0, "language": [], "certification_needed": false, "location_preference": "", "completion_percent": 55}
🛠️ Tool Response -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Result: {"percent_complete":

In [23]:
turn_2 = swarm.invoke(
    {"messages": [{"role": "user", "content": "I prefer English"}]},
    config,
)
# print(turn_2["messages"][-1].content)

pretty_print_turn(turn_2)

/var/folders/hq/tbxvw3vn6wb7ky65zh2vrfm40000gn/T/ipykernel_30364/1747705358.py:45: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  new_profile = partial_profile_response.dict()



=== Conversation Turn ===
🧑 Human: Hi
🤖 AI: Hello! How can I assist you today? Are you looking for information on courses or programs?
🧑 Human: I'm Tejas Gadi, 20 years old, a high school student interested in Maths 
🤖 AI: 
🔧 Tool Call -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Name: extract_student_profile | Args: {'conversation': "I'm Tejas Gadi, 20 years old, a high school student interested in Maths."}
🔧 Tool Call -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Name: check_profile_completeness | Args: {'description': "Check the completeness of Tejas's student profile."}
🛠️ Tool Response -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Result: {"name": "Tejas Gadi", "educational_level": "High School", "age": 20, "course_description": "", "course_mode": "", "daily_hours": 0, "preferred_timing": "", "max_duration_months": 0, "language": [], "certification_needed": false, "location_preference": "", "completion_percent": 55}
🛠️ Tool Response -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Result: {"percent_complete":

In [24]:
turn_2 = swarm.invoke(
    {"messages": [{"role": "user", "content": "No preferences for location?"}]},
    config,
)
# print(turn_2["messages"][-1].content)

pretty_print_turn(turn_2)

/var/folders/hq/tbxvw3vn6wb7ky65zh2vrfm40000gn/T/ipykernel_30364/1747705358.py:45: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  new_profile = partial_profile_response.dict()



=== Conversation Turn ===
🧑 Human: Hi
🤖 AI: Hello! How can I assist you today? Are you looking for information on courses or programs?
🧑 Human: I'm Tejas Gadi, 20 years old, a high school student interested in Maths 
🤖 AI: 
🔧 Tool Call -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Name: extract_student_profile | Args: {'conversation': "I'm Tejas Gadi, 20 years old, a high school student interested in Maths."}
🔧 Tool Call -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Name: check_profile_completeness | Args: {'description': "Check the completeness of Tejas's student profile."}
🛠️ Tool Response -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Result: {"name": "Tejas Gadi", "educational_level": "High School", "age": 20, "course_description": "", "course_mode": "", "daily_hours": 0, "preferred_timing": "", "max_duration_months": 0, "language": [], "certification_needed": false, "location_preference": "", "completion_percent": 55}
🛠️ Tool Response -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Result: {"percent_complete":

In [25]:
turn_2 = swarm.invoke(
    {"messages": [{"role": "user", "content": "Yes please"}]},
    config,
)
# print(turn_2["messages"][-1].content)

pretty_print_turn(turn_2)


=== Conversation Turn ===
🧑 Human: Hi
🤖 AI: Hello! How can I assist you today? Are you looking for information on courses or programs?
🧑 Human: I'm Tejas Gadi, 20 years old, a high school student interested in Maths 
🤖 AI: 
🔧 Tool Call -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Name: extract_student_profile | Args: {'conversation': "I'm Tejas Gadi, 20 years old, a high school student interested in Maths."}
🔧 Tool Call -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Name: check_profile_completeness | Args: {'description': "Check the completeness of Tejas's student profile."}
🛠️ Tool Response -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Result: {"name": "Tejas Gadi", "educational_level": "High School", "age": 20, "course_description": "", "course_mode": "", "daily_hours": 0, "preferred_timing": "", "max_duration_months": 0, "language": [], "certification_needed": false, "location_preference": "", "completion_percent": 55}
🛠️ Tool Response -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Result: {"percent_complete":

In [26]:
turn_2 = swarm.invoke(
    {"messages": [{"role": "user", "content": "Okay"}]},
    config,
)
# print(turn_2["messages"][-1].content)

pretty_print_turn(turn_2)


=== Conversation Turn ===
🧑 Human: Hi
🤖 AI: Hello! How can I assist you today? Are you looking for information on courses or programs?
🧑 Human: I'm Tejas Gadi, 20 years old, a high school student interested in Maths 
🤖 AI: 
🔧 Tool Call -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Name: extract_student_profile | Args: {'conversation': "I'm Tejas Gadi, 20 years old, a high school student interested in Maths."}
🔧 Tool Call -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Name: check_profile_completeness | Args: {'description': "Check the completeness of Tejas's student profile."}
🛠️ Tool Response -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Result: {"name": "Tejas Gadi", "educational_level": "High School", "age": 20, "course_description": "", "course_mode": "", "daily_hours": 0, "preferred_timing": "", "max_duration_months": 0, "language": [], "certification_needed": false, "location_preference": "", "completion_percent": 55}
🛠️ Tool Response -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Result: {"percent_complete":

In [27]:
turn_2 = swarm.invoke(
    {"messages": [{"role": "user", "content": "Where are my recommendations on course?"}]},
    config,
)
# print(turn_2["messages"][-1].content)

pretty_print_turn(turn_2)


=== Conversation Turn ===
🧑 Human: Hi
🤖 AI: Hello! How can I assist you today? Are you looking for information on courses or programs?
🧑 Human: I'm Tejas Gadi, 20 years old, a high school student interested in Maths 
🤖 AI: 
🔧 Tool Call -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Name: extract_student_profile | Args: {'conversation': "I'm Tejas Gadi, 20 years old, a high school student interested in Maths."}
🔧 Tool Call -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Name: check_profile_completeness | Args: {'description': "Check the completeness of Tejas's student profile."}
🛠️ Tool Response -> ID: call_bOhtHodywDVNjST6yYajzWl3 | Result: {"name": "Tejas Gadi", "educational_level": "High School", "age": 20, "course_description": "", "course_mode": "", "daily_hours": 0, "preferred_timing": "", "max_duration_months": 0, "language": [], "certification_needed": false, "location_preference": "", "completion_percent": 55}
🛠️ Tool Response -> ID: call_spUN8pvSs5sW5JhzXoK0FnR2 | Result: {"percent_complete":

In [ ]:
turn_2 = swarm.invoke(
    {"messages": [{"role": "user", "content": "What are the career opportunities/career paths in Maths and Computer Science"}]},
    config,
)
# print(turn_2["messages"][-1].content)

pretty_print_turn(turn_2)